In [1]:
from IPython.core.display import display, HTML,display_html
from IPython.display import display, Markdown, clear_output
from ipywidgets import interact, interact_manual
import ipywidgets as widgets
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import seaborn as sns
%load_ext autoreload
%autoreload 2

Loading Libs

In [2]:
df = pd.read_csv('./DataFrames/qbinsModel.csv')

In [3]:
df = df[['riskBad','bad_90','riskGood']]
df.columns = ['riskBad','bad','riskGood']

In [4]:
qbins =20
out, bins = pd.qcut(df['riskGood'],qbins,duplicates='drop', retbins=True)

### Create Groups

In [185]:
def get_bins(x,binz):
    for i in range(len(binz)):
        if i == 0:
            if x <= binz[i]:
                return i
        elif i == len(binz)-1:
            if x >= binz[i]:
                return i+1
            elif x >= binz[i-1] and x < binz[i]:
                return i
        else:
            if x >= binz[i-1] and x < binz[i]:
                return i

In [186]:
def getRating(x):
    rating = ['A1','A2','A3','B1','B2','B3','C1','C2','C3','D1','D2','D3','E','F']
    thresh = [0.03,0.06,0.08,0.1,0.15,0.2,0.25,0.35,0.5,0.6,0.7,0.8,0.9]
    for i in range(len(thresh)):
        if i == 0:
            if x <= thresh[i]:
                return rating[i]
        elif i == len(thresh)-1:
            if x >= thresh[i]:
                return rating[i+1]
            elif x >= thresh[i-1] and x < thresh[i]:
                return rating[i]
        else:
            if x >= thresh[i-1] and x < thresh[i]:
                return rating[i]

In [187]:
def createRiskGroup(df,binz):
    groups = []
    for i in range(1,len(binz)+1):
        try:
            groups.append({
                'group' : i, 'threshold' : binz[i-1] , 
                'bad(%)' : df['bad'][df['risk_group'] == i].value_counts(normalize=True).sort_index().values[1]*100 , 
                'pop(%)' : (df[df['risk_group'] == i].shape[0]/df.shape[0])*100 , 
                'rating' : getRating(df['bad'][df['risk_group'] == i].value_counts(normalize=True).sort_index().values[1]) 
            })
        except:
            groups.append({
                'group' : i, 'threshold' : binz[i-1] , 
                'bad(%)' : 0, 
                'pop(%)' : (df[df['risk_group'] == i].shape[0]/df.shape[0])*100, 
                'rating' : getRating(0) 
            })
    return groups

In [188]:
df['risk_group'] = df['riskGood'].map(lambda x: get_bins(x,bins[1:-1]))
grouped = pd.DataFrame(createRiskGroup(df,bins[1:-1]))

In [189]:
grouped

,group,threshold,bad(%),pop(%),rating
0,1,0.002564,0.000000,0.000000,A1
1,2,0.010309,98.969072,1.835731,F
2,3,0.035316,96.468401,10.181681,F
3,4,0.063927,92.881356,5.582892,F
4,5,0.114525,88.547486,6.775170,E
5,6,0.151079,84.012539,6.037093,E
6,7,0.197368,76.744186,4.882665,D3
7,8,0.333333,60.425532,4.447388,D2
8,9,0.500000,50.000000,0.870553,D1
9,10,0.534247,43.644068,8.932627,C3


### Decision Process

In [190]:
df['risk_group'] = df['riskGood'].map(lambda x: get_bins(x,bins[1:-1]))
grouped = pd.DataFrame(createRiskGroup(df,bins[1:-1]))

changes = 1
while changes != 0:
    newBins = list(grouped['threshold'].values)
    changes = 0
    i = 0
    # kinda recursive
    while i <= grouped.shape[0]-2:
        
        if grouped.loc[i+1,'bad(%)'] > 35:
            
            if grouped.loc[i,'rating'] == grouped.loc[i+1,'rating']:
                
                newBins.remove(grouped.loc[i,'threshold'])
                i = i + 1
                changes = changes + 1
            
            elif np.abs(grouped.loc[i,'bad(%)'] - grouped.loc[i+1,'bad(%)']) < 15 and (grouped.loc[i,'pop(%)'] + grouped.loc[i+1,'pop(%)']) < 15 and grouped.loc[i+1,'bad(%)'] > 2:
                
                #print('3 - Dropping',str(grouped.loc[i,'threshold']),'Leaving',str(grouped.loc[i+1,'threshold']))
                
                newBins.remove(grouped.loc[i,'threshold'])
                i = i + 1
                changes = changes + 1
                
            else:
                pass
                
        else:
            
            if grouped.loc[i,'rating'] == grouped.loc[i+1,'rating']:
                
                newBins.remove(grouped.loc[i,'threshold'])
                i = i + 1
                changes = changes + 1
            
            elif (grouped.loc[i,'pop(%)']) < 2 and np.abs(grouped.loc[i,'bad(%)'] - grouped.loc[i+1,'bad(%)']) < grouped.loc[i+1,'bad(%)']/10 and grouped.loc[i+1,'bad(%)'] < 2:
                
                print('0 - Dropping',str(grouped.loc[i,'threshold']),'Leaving',str(grouped.loc[i+1,'threshold']))
                
                newBins.remove(grouped.loc[i,'threshold'])
                i = i + 1
                changes = changes + 1
                
            elif np.abs(grouped.loc[i,'bad(%)'] - grouped.loc[i+1,'bad(%)']) < grouped.loc[i+1,'bad(%)']/10 and (grouped.loc[i,'pop(%)'] + grouped.loc[i+1,'pop(%)']) < 10 and  grouped.loc[i+1,'bad(%)'] > 2:
                
                print('1 - Dropping',str(grouped.loc[i,'threshold']),'Leaving',str(grouped.loc[i+1,'threshold']))
                
                newBins.remove(grouped.loc[i,'threshold'])
                i = i + 1
                changes = changes + 1
                
            else:
                pass
        
        if i <= grouped.shape[0]:
            i = i + 1
        
    newBins = sorted(list(set(newBins)))
    df['risk_group'] = df['riskGood'].map(lambda x: get_bins(x,newBins))
    grouped = pd.DataFrame(createRiskGroup(df,newBins))
    #display(grouped)
grouped

,group,threshold,bad(%),pop(%),rating
0,1,0.035316,92.881356,5.582892,F
1,2,0.114525,86.410635,12.812263,E
2,3,0.197368,67.346939,10.200606,D2
3,4,0.534247,43.644068,8.932627,C3
4,5,0.732323,25.547445,5.185466,C2
5,6,0.818182,16.738197,4.409538,B3
6,7,0.911111,7.590759,5.734292,A3
7,8,0.944444,3.910615,6.775170,A2
8,9,0.995413,0.215517,8.781226,A1


In [215]:
def printThresholds(binz):
    
    for i in range(len(binz)):
        if i == 0:
            print('if x <=',np.round(binz['threshold'][i+1],5),':')
            print('\treturn',i+1,',"'+binz['rating'][i]+'"')
        elif i == len(binz)-1:
            print('elif x > ',np.round(binz['threshold'][i],5),':')
            print('\treturn',i+1,',"'+binz['rating'][i]+'"')
        else:
            print('elif x > ',np.round(binz['threshold'][i],5),'and  x <= ',np.round(binz['threshold'][i+1],4),':')
            print('\treturn',i+1,',"'+binz['rating'][i]+'"')

In [216]:
printThresholds(grouped[['threshold','rating']].sort_values('threshold'))

if x <= 0.11453 :
	return 1 ,"F"
elif x >  0.11453 and  x <=  0.1974 :
	return 2 ,"E"
elif x >  0.19737 and  x <=  0.5342 :
	return 3 ,"D2"
elif x >  0.53425 and  x <=  0.7323 :
	return 4 ,"C3"
elif x >  0.73232 and  x <=  0.8182 :
	return 5 ,"C2"
elif x >  0.81818 and  x <=  0.9111 :
	return 6 ,"B3"
elif x >  0.91111 and  x <=  0.9444 :
	return 7 ,"A3"
elif x >  0.94444 and  x <=  0.9954 :
	return 8 ,"A2"
elif x >  0.99541 :
	return 9 ,"A1"
